In [1]:
import numpy as np
from typing import Tuple
import scipy.constants as np
import h5py

Be aware of some lazy coding. DO not copy this stupidly into the main program

In [ ]:
def load_data(filename:str) -> Tuple[np.ndarray, np.ndarray, float, np.ndarray, np.ndarray]:
    """Loads the datafile indicated and returns basic properties

    Args:
        filename (str): path to the .hdf5 file

    Returns:
        Tuple:
            pos (np.ndarray):
            force (np.ndarray):
            L (float):
            N (np.ndarray):
            t (np.ndarray):
    """
    df = h5py.File(filename)
    skips =  df['/input/incar/ML_OUTBLOCK'][()]
    L = df['results/positions/lattice_vectors'][()][0, 0]
    pos = df['intermediate/ion_dynamics/position_ions'][()]*L
    force = df['intermediate/ion_dynamics/forces'][()]
    N = df['results/positions/number_ion_types'][()]
    dt = df['/input/incar/POTIM'][()]*skips
    t =np.arange(pos.shape[0])*dt
    return pos, force, L, N, t

def interparticle_dis(i:int, pos:np.ndarray, N:np.ndarray):
    # dumly load some properties
    N_H = N[0]
    N_O = N[1]
    N_K = N[2]
    N_OH = N_K  # intended number of OH- from input file
    N_H2O = N_O - N_OH
    H_i = np.arange(N_H)
    O_i  = np.arange(N_H, N_H + N_O)
    K_i = np.arange(N_H + N_O, N_H + N_O + N_K)

    # prep interparticle arrays
    pos_H = pos[:, H_i, :]
    pos_O = pos[:, O_i, :]
    pos_K = pos[:, K_i, :]
    